In [208]:
class Seedstock
  require 'csv'
  attr_accessor :table
  attr_accessor :stockID
  attr_accessor :mutant
  attr_accessor :path
  
  def initialize (stockpath:)
    unless File.file?(stockpath)
      return ("File doesn't exist")
    else
      @path = Dir.pwd
      @stockpath = stockpath
      @stockID = stockID
      @mutant = mutant
      @table = CSV.read(File.open(stockpath), headers: true, col_sep: "\t")
    end
  end


  def seeding (seednum)
    path = @path
    newstock = @table
    newstock.each { |row| #Iterating over each row
      row[4] = (row[4].to_i - seednum) # Substraction of input from Grams_Remaining
      if row[4] <= 0 # return error message if value < 0
        row[4]=0
        $stderr.puts "WARNING: we have run out of Seed Stock #{row[0]}"
      end
    }
    CSV.open("#{path}/newstock.tsv", 'w', col_sep: "\t") do |tsv|
      tsv << newstock.headers
      newstock.each { |row| tsv << row }
    end
    return newstock
  end
  
  def linking (stockid, mut_gene)
    @stockID = stockid
    @mutant = mut_gene
  end
    
end

#Testing
#stock = Seedstock.new(stockpath: "/home/osboxes/bioinfogit/bioinfo_Chals/seed_stock_data.tsv")
#puts stock.table 
#puts stock.seeding(7)

Seed_Stock,Mutant_Gene_ID,Last_Planted,Storage,Grams_Remaining
A334,AT1G69120,5/7/2014,cama2,28
A348,AT4G36920,3/11/2013,cama25,12
B3334,AT3G54340,1/12/2014,cama18,22
A51,AT1G30950,9/2/2013,cama25,5
B52,AT5G20240,8/11/2012,cama16,7


Seed_Stock,Mutant_Gene_ID,Last_Planted,Storage,Grams_Remaining
A334,AT1G69120,5/7/2014,cama2,21
A348,AT4G36920,3/11/2013,cama25,5
B3334,AT3G54340,1/12/2014,cama18,15
A51,AT1G30950,9/2/2013,cama25,0
B52,AT5G20240,8/11/2012,cama16,0
